# Пользовательские демографические группы
Ноутбук с примерами формирования пользовательских демографических групп.

Общие принципы формирования демографических групп:
- формируются на основе значений справочника социально-демографических переменных
- доступные операторы:
    ```
    = (равенство)
    != (неравенство)
    > (больше)
    < (меньше)
    >= (больше или равно) 
    <= (меньше или равно)
    AND (И)
    OR (ИЛИ)
    IN() (вхождение в список значений)
    ```

Рассмотрим подробнее на примере следующих демографических групп:
1. Мужчины 35-44 лет
2. Мужчины 18-24 лет или женщины 25-34 лет
3. Все 20-56 лет
4. Не руководители 25-54 лет
5. Работающие жители Большой Москвы

Параметры:
- Период: Сентябрь 2021
- Тип пользования интернетом: ограничения нет, считаем по всем (Web Desktop, Web Mobile, App Mobile)
- Ресурс: Ivi

Статистики:
- Reach (reach)

# Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к API Cross Web и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
#import matplotlib.pyplot as plt
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.crossweb import tasks as cwt
from mediascope_api.crossweb import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Создаем объекты для работы с API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.CrossWebTask()
cats = cwc.CrossWebCats()

## Список социально-демографических переменных
Для формирования синтаксиса демографических групп нам понадобится справочник социально-демографических переменных.

Найдем необходимые социально-демографические переменные с помощью метода поиска в справочнике. Работа со справочником подробно описана в ноутбуке [catalogs](catalogs.ipynb). 

In [ ]:
# Например, найдем в справочнике переменные, содержащие текст "Пол"
cats.find_property('Пол', expand=True)

### Общие параметры для заданий

Для начала зададим общие параметры

In [ ]:
# Задаем период
# Период указывается в виде списка ('Начало', 'Конец'). Можно указать несколько периодов
date_filter = [('2021-09-01', '2021-09-30')]

# Задаем фильтр по типам пользования интернетом
usetype_filter = [1,2,3]

## Задания

Перейдем к формированию заданий.


### Задание №1. Мужчины 35-44 лет

Из справочника видим, что для отбора мужчин нам нужно значение 1 переменной **sex**
```
    sex = 1
```
Возрастной группе 35-44 соответствует значение 4 переменной **ageGroup**
```
    ageGroup = 4
```
Объединяем выражения через оператор AND
```
    sex = 1 AND ageGroup = 4
```

### Получим ID ресурса
Для построения отчета необходимо получить идентификатор ресурса __Ivi__.

Для этого воспользуемся методом поиска в медиа справочнике. Работа с медиа справочником подробно описана в ноутбуке [catalogs](catalogs.ipynb). 

Получим идентификатор ресурса **Ivi**

In [ ]:
cats.get_resource(resource='Ivi')

Таким образом, необходимый идентификатор следующий:

- **Ivi**  resourceId = 1067

In [ ]:
# Задаем название для отображения в DataFrame
project_name = 'Ivi M35-44'

# Задаем фильтр по географии, в нашем случае он не требуется
geo_filter = None

# Задаем фильтр по демографии, в нашем случае составленное выше выражение для переменной "Мужчины 35-44 лет" 
demo_filter = 'sex = 1 AND ageGroup = 4'

# Задаем фильтр по медиа, в нашем случае это ID ресурса Ivi
mart_filter = 'crossMediaResourceId = 1067'

# Указываем список срезов, чтобы сформировать структуру расчета
slices = ["researchMonth", "crossMediaResourceId"]

# Указываем список статистик для расчета
statistics = ['reach']

# Формируем задание для API Cross Web в формате JSON
task_json = mtask.build_task('media', project_name, date_filter, usetype_filter, geo_filter,
                             demo_filter, mart_filter, slices, statistics)

# Отправляем задание на расчет и ждем выполнения
task_audience = mtask.wait_task(mtask.send_audience_task(task_json))

# Получаем результат
df_1 = mtask.result2table(mtask.get_result(task_audience),project_name)
df_1

### Задание №2. Мужчины 18-24 лет или женщины 25-34 лет

Возвращаемся к справочнику социально-демографических переменных. Для отбора мужчин 18-24 лет нам нужно значение 1 переменной **sex** и значение 2 переменной **ageGroup**
```
sex = 1 AND ageGroup = 2
```
Для отбора женщин 25-34 лет синтаксис будет следующим:
```
sex = 2 AND ageGroup = 3
```
Объединяем через оператор OR
```
(sex = 1 AND ageGroup = 2) OR (sex = 2 AND ageGroup = 3)
```

In [ ]:
# Задаем название для отображения в DataFrame
project_name = 'Ivi M18-24 + W25-34'

# Задаем фильтр по географии, в нашем случае он не требуется
geo_filter = None

# Задаем фильтр по демографии, в нашем случае составленное выше выражение для переменной "Мужчины 18-24 лет или женщины 25-34 лет"
demo_filter = '(sex = 1 AND ageGroup = 2) OR (sex = 2 AND ageGroup = 3)'

# Задаем фильтр по медиа, в нашем случае это ID ресурса Ivi
mart_filter = 'crossMediaResourceId = 1067'

# Указываем список срезов, чтобы сформировать структуру расчета
slices = ["researchMonth", "crossMediaResourceId"]

# Указываем список статистик для расчета
statistics = ['reach']

# Формируем задание для API Cross Web в формате JSON
task_json = mtask.build_task('media', project_name, date_filter, usetype_filter, geo_filter,
                             demo_filter, mart_filter, slices, statistics)

# Отправляем задание на расчет и ждем выполнения
task_audience = mtask.wait_task(mtask.send_audience_task(task_json))

# Получаем результат
df_2 = mtask.result2table(mtask.get_result(task_audience), project_name)
df_2

### Задание №3. Все 20-56 лет

Возраст 20-56 лет не получится собрать из переменной **ageGroup**, нам понадобится переменная **age**.

Зададим диапазон возраста с помощью операторов сравнения "больше или равно" (**>=**) и "меньше или равно" (**<=**):
```
age >= 20 AND age <= 56
```

In [ ]:
# Задаем название для отображения в DataFrame
project_name = 'Ivi All 20-56'

# Задаем фильтр по географии, в нашем случае он не требуется
geo_filter = None

# Задаем фильтр по демографии, в нашем случае составленное выше выражение для переменной "Все 20-56 лет"
demo_filter = 'age >= 20 AND age <= 56'

# Задаем фильтр по медиа, в нашем случае это ID ресурса Ivi
mart_filter = 'crossMediaResourceId = 1067'

# Указываем список срезов, чтобы сформировать структуру расчета
slices = ["researchMonth", "crossMediaResourceId"]

# Указываем список статистик для расчета
statistics = ['reach']

# Формируем задание для API Cross Web в формате JSON
task_json = mtask.build_task('media', project_name, date_filter, usetype_filter, geo_filter,
                             demo_filter, mart_filter, slices, statistics)

# Отправляем задание на расчет и ждем выполнения
task_audience = mtask.wait_task(mtask.send_audience_task(task_json))

# Получаем результат
df_3 = mtask.result2table(mtask.get_result(task_audience), project_name)
df_3

### Задание №4. Не руководители 25-54 лет

Для отбора возрастной группы 25-54 нам понадобятся значения 3, 4 и 5 переменной **ageGroup**. Для упрощения мы не будем составлять три условия через OR, а воспользуемся оператором IN
```
ageGroup IN (3, 4, 5)
```
Условие "не руководители" можно также коротко задать при помощи оператора неравенства "!="
```
occupation != 1
```
Объединяем через оператор AND
```
ageGroup IN (3, 4, 5) AND occupation != 1
```

In [ ]:
# Задаем название для отображения в DataFrame
project_name = 'Ivi 25-54 not dir'

# Задаем фильтр по географии, в нашем случае он не требуется
geo_filter = None

# Задаем фильтр по демографии, в нашем случае составленное выше выражение для переменной "Не руководители 25-54 лет"
demo_filter = 'ageGroup IN (3, 4, 5) AND occupation != 1'

# Задаем фильтр по медиа, в нашем случае это ID ресурса Ivi
mart_filter = 'crossMediaResourceId = 1067'

# Указываем список срезов, чтобы сформировать структуру расчета
slices = ["researchMonth", "crossMediaResourceId"]

# Указываем список статистик для расчета
statistics = ['reach']

# Формируем задание для API Cross Web в формате JSON
task_json = mtask.build_task('media', project_name, date_filter, usetype_filter, geo_filter,
                             demo_filter, mart_filter, slices, statistics)

# Отправляем задание на расчет и ждем выполнения
task_audience = mtask.wait_task(mtask.send_audience_task(task_json))

# Получаем результат
df_4 = mtask.result2table(mtask.get_result(task_audience), project_name)
df_4

### Задание №5. Работающие жители Большой Москвы

Географию "Большая Москва" зададим при помощи переменной **сityPop**
```
cityPop = 1
```
Для выбора работающих нам потребуется переменная **work**
```
work = 1
```

In [ ]:
# Задаем название для отображения в DataFrame
project_name = 'Ivi BM work'

# Задаем фильтр по географии, в нашем случае составленное выше выражение для Большой Москвы
geo_filter = 'cityPop = 1'

# Задаем фильтр по демографии, в нашем случае составленное выше выражение для работающих 
demo_filter = 'work = 1'

# Задаем фильтр по медиа, в нашем случае это ID ресурса Ivi
mart_filter = 'crossMediaResourceId = 1067'

# Указываем список срезов, чтобы сформировать структуру расчета
slices = ["researchMonth", "crossMediaResourceId"]

# Указываем список статистик для расчета
statistics = ['reach']

# Формируем задание для API Cross Web в формате JSON
task_json = mtask.build_task('media', project_name, date_filter, usetype_filter, geo_filter,
                             demo_filter, mart_filter, slices, statistics)

# Отправляем задание на расчет и ждем выполнения
task_audience = mtask.wait_task(mtask.send_audience_task(task_json))

# Получаем результат
df_5 = mtask.result2table(mtask.get_result(task_audience), project_name)
df_5